In [2]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, SVR
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, \
RocCurveDisplay, roc_auc_score, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import VotingClassifier, VotingRegressor, BaggingClassifier

In [3]:
hr = pd.read_csv('C:/Users/dbda.STUDENTSDC/Desktop/M.L/Datasets/HR_comma_sep.csv')
hr.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low


In [ ]:
x,y = hr.drop('left',axis=1),hr['left']
ohe = OneHotEncoder(drop='first',sparse_output=False).set_output(transform='pandas')

from sklearn.compose import ColumnTransformer, make_column_selector

col_trnf = ColumnTransformer([
                                ('OHE', ohe, make_column_selector(dtype_include=object))],
                              remainder='passthrough', verbose_feature_names_out=False)

col_trnf = col_trnf.set_output(transform='pandas')
x = col_trnf.fit_transform(x)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=25,stratify=y)

In [7]:
dtc1 = DecisionTreeClassifier(random_state=25,max_depth=4)
dtc2 = DecisionTreeClassifier(random_state=25,max_depth=2)

lr = LogisticRegression(max_iter=1000)

scaler = StandardScaler()
knn = KNeighborsClassifier(n_neighbors=5)
pipe_knn = Pipeline([('SCL',scaler),('KNN',knn)])

svm = SVC(kernel='rbf',probability=True,random_state=25)
pipe_svm = Pipeline([('SCL',scaler),('SVM',svm)])

estimators = [dtc1,dtc2, lr,pipe_knn,pipe_svm]
scores = []

for i in estimators:
    bagg = BaggingClassifier(random_state=25, estimator=i, n_estimators=15)
    bagg.fit(x_train,y_train)
    y_pred = bagg.predict(x_test)
    scores.append([i, accuracy_score(y_test, y_pred)])

df_score = pd.DataFrame(scores, columns=['Base Model', 'Accuracy Score'])
df_score.sort_values('Accuracy Score', ascending=False)

,Base Model,Accuracy Score
0,"DecisionTreeClassifier(max_depth=4, random_sta...",0.969327
4,"(StandardScaler(), SVC(probability=True, rando...",0.951767
3,"(StandardScaler(), KNeighborsClassifier())",0.944877
1,"DecisionTreeClassifier(max_depth=2, random_sta...",0.850189
2,LogisticRegression(max_iter=1000),0.797733


In [8]:
os.chdir('C:/Users/dbda.STUDENTSDC/Desktop/M.L/Datasets')
boston = pd.read_csv('Boston.csv')

boston

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [9]:
x, y = boston.drop('medv', axis = 1), boston.medv

# ss = StandardScaler().set_output(transform='pandas')
# x = ss.fit_transform(x)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=25)

In [11]:
from sklearn.ensemble import BaggingRegressor


svm = SVR(kernel = 'rbf')
scaler = StandardScaler()
knn = KNeighborsRegressor(n_neighbors=5)
pipe_svm = Pipeline([('Scaling', scaler),('SVM', svm)])
pipe_knn = Pipeline([('Scaling', scaler),('KNN', knn)])
dtc1 = DecisionTreeRegressor(random_state=25, max_depth=4)
dtc2 = DecisionTreeRegressor(random_state=25)

estimators = [dtc1, dtc2, knn, pipe_knn, pipe_svm]
scores = []
for e in tqdm(estimators):
    bagg = BaggingRegressor(random_state=25, estimator = e, n_estimators=15)
    bagg.fit(x_train, y_train)
    y_pred = bagg.predict(x_test)
    scores.append([e,mean_absolute_error(y_test, y_pred)])
    # print(accuracy_score(y_test, y_pred))
df_scores = pd.DataFrame(scores , columns = ['Estimator', 'mean_absolute_error'])
df_scores.sort_values('mean_absolute_error', ascending=True)

100%|██████████| 5/5 [00:00<00:00,  8.41it/s]


,Estimator,mean_absolute_error
1,DecisionTreeRegressor(random_state=25),2.325439
0,"DecisionTreeRegressor(max_depth=4, random_stat...",2.540984
3,"(StandardScaler(), KNeighborsRegressor())",2.722079
4,"(StandardScaler(), SVR())",3.014861
2,KNeighborsRegressor(),4.316605


On Concrete DAta (BAgging)

In [14]:
concrete = pd.read_csv('C:/Users/dbda.STUDENTSDC/Desktop/M.L/Cases/Concrete_Strength/Concrete_Data.csv')
concrete.head()

,Cement,Blast,Fly,Water,Superplasticizer,Coarse,Fine,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [17]:
x,y = concrete.drop("Strength",axis=1),concrete["Strength"]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=25)

In [18]:
estimators = [dtc1, dtc2, knn, pipe_knn, pipe_svm]
n_est = [10,25,50,100]
scores = []
for e in tqdm(estimators):
    for n in n_est:
        bagg = BaggingRegressor(random_state=25, estimator = e, n_estimators=n)
        bagg.fit(x_train, y_train)
        y_pred = bagg.predict(x_test)
        scores.append([e,n,mean_absolute_error(y_test, y_pred)])
        # print(accuracy_score(y_test, y_pred))
df_scores = pd.DataFrame(scores , columns = ['Estimator','n', 'score'])
df_scores.sort_values('score')

100%|██████████| 5/5 [00:11<00:00,  2.20s/it]


,Estimator,n,score
7,DecisionTreeRegressor(random_state=25),100,3.755561
6,DecisionTreeRegressor(random_state=25),50,3.774158
5,DecisionTreeRegressor(random_state=25),25,3.838482
4,DecisionTreeRegressor(random_state=25),10,3.891324
1,"DecisionTreeRegressor(max_depth=4, random_stat...",25,6.407477
0,"DecisionTreeRegressor(max_depth=4, random_stat...",10,6.430295
12,"(StandardScaler(), KNeighborsRegressor())",10,6.505643
3,"DecisionTreeRegressor(max_depth=4, random_stat...",100,6.532116
15,"(StandardScaler(), KNeighborsRegressor())",100,6.544374
13,"(StandardScaler(), KNeighborsRegressor())",25,6.559220


In [19]:
from sklearn.linear_model import ElasticNet, LinearRegression, Ridge


lr,ridge,el = LinearRegression(),Ridge(),ElasticNet()
estimators = [lr,ridge,el]
n_est = [10,25,50,100]
scores = []
for e in tqdm(estimators):
    for n in n_est:
        bagg = BaggingRegressor(random_state=25, estimator = e, n_estimators=n)
        bagg.fit(x_train, y_train)
        y_pred = bagg.predict(x_test)
        scores.append([e,n,mean_absolute_error(y_test, y_pred)])
        # print(accuracy_score(y_test, y_pred))
df_scores = pd.DataFrame(scores , columns = ['Estimator','n', 'score'])
df_scores.sort_values('score')

100%|██████████| 3/3 [00:01<00:00,  1.97it/s]


,Estimator,n,score
0,LinearRegression(),10,7.675439
4,Ridge(),10,7.675441
8,ElasticNet(),10,7.682072
3,LinearRegression(),100,7.711915
7,Ridge(),100,7.711917
1,LinearRegression(),25,7.714196
5,Ridge(),25,7.714198
11,ElasticNet(),100,7.718568
9,ElasticNet(),25,7.722113
2,LinearRegression(),50,7.724467
